<a href="https://colab.research.google.com/github/A-R-M-028/Autoencoders-GAN---Deep-Fake/blob/main/D54_Autoencoders_%26_GAN_Deep_Fake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
%cd first-order-model
!pip install ffmpeg-python
!pip install imageio
!pip install imageio-ffmpeg

Cloning into 'first-order-model'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 393 (delta 41), reused 64 (delta 38), pack-reused 312
Receiving objects: 100% (393/393), 72.19 MiB | 31.07 MiB/s, done.
Resolving deltas: 100% (204/204), done.
/content/first-order-model


In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
                            checkpoint_path='/content/drive/MyDrive/Datasets/GAN/Copy of vox-cpk.pth.tar')

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

source_image = imageio.imread('/content/drive/MyDrive/Datasets/GAN/Copy of Emily.png')
reader = imageio.get_reader('/content/drive/MyDrive/Datasets/GAN/Copy of driver.mp4')

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)
#save resulting video
imageio.mimsave('../output.mp4', [img_as_ubyte(frame) for frame in predictions])

HTML(display(source_image, driving_video, predictions).to_html5_video())

  0%|          | 0/852 [00:00<?, ?it/s]